In [1]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow
import pandas as pd
from matplotlib import pyplot as plt
import ast
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from joblib import dump
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def flatten_keypoints(df, keypoint_columns):
    for col in keypoint_columns:
        df[col + '_x'] = df[col].apply(lambda x: x[0])
        df[col + '_y'] = df[col].apply(lambda x: x[1])
    # Drop the original columns
    df.drop(columns=keypoint_columns, inplace=True)


# SVM MoveNet LightningModel


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/labeled_keypointsWithoutScore.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11723 entries, 0 to 11722
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   video_id                         11723 non-null  int64 
 1   frame_idx                        11723 non-null  int64 
 2   label                            11723 non-null  int64 
 3   MoveNetLightning_nose            11723 non-null  object
 4   MoveNetLightning_left_eye        11723 non-null  object
 5   MoveNetLightning_right_eye       11723 non-null  object
 6   MoveNetLightning_left_ear        11723 non-null  object
 7   MoveNetLightning_right_ear       11723 non-null  object
 8   MoveNetLightning_left_shoulder   11723 non-null  object
 9   MoveNetLightning_right_shoulder  11723 non-null  object
 10  MoveNetLightning_left_elbow      11723 non-null  object
 11  MoveNetLightning_right_elbow     11723 non-null  object
 12  MoveNetLightning_left_wrist     

In [ ]:
keypoint_columns = ['MoveNetLightning_nose', 'MoveNetLightning_left_eye', 'MoveNetLightning_right_eye',
                    'MoveNetLightning_left_ear', 'MoveNetLightning_right_ear',
                    'MoveNetLightning_left_shoulder', 'MoveNetLightning_right_shoulder',
                    'MoveNetLightning_left_elbow', 'MoveNetLightning_right_elbow',
                    'MoveNetLightning_left_wrist', 'MoveNetLightning_right_wrist',
                    'MoveNetLightning_left_hip', 'MoveNetLightning_right_hip',
                    'MoveNetLightning_left_knee', 'MoveNetLightning_right_knee',
                    'MoveNetLightning_left_ankle', 'MoveNetLightning_right_ankle']
for col in keypoint_columns:
    df[col] = df[col].apply(literal_eval)



In [ ]:

# Define your features and target
X = df[keypoint_columns]
y = df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)


##SVM and Data Ready For Training

In [ ]:
# Apply flattening to both X_train and X_test
flatten_keypoints(X_train, keypoint_columns)
flatten_keypoints(X_test, keypoint_columns)

In [ ]:
df.head()

,video_id,frame_idx,label,MoveNetLightning_nose,MoveNetLightning_left_eye,MoveNetLightning_right_eye,MoveNetLightning_left_ear,MoveNetLightning_right_ear,MoveNetLightning_left_shoulder,MoveNetLightning_right_shoulder,MoveNetLightning_left_elbow,MoveNetLightning_right_elbow,MoveNetLightning_left_wrist,MoveNetLightning_right_wrist,MoveNetLightning_left_hip,MoveNetLightning_right_hip,MoveNetLightning_left_knee,MoveNetLightning_right_knee,MoveNetLightning_left_ankle,MoveNetLightning_right_ankle
0,1,0,0,"[0.95387256, 0.46946964]","[0.9538181, 0.4650093]","[0.9537934, 0.4664463]","[0.95696807, 0.468577]","[0.95457935, 0.4690094]","[0.7321407, 0.4456314]","[0.51548225, 0.3184227]","[0.9233335, 0.6233904]","[0.46472308, 0.49570796]","[0.67127454, 0.44328585]","[0.45589715, 0.4999305]","[0.5438332, 0.5275059]","[0.46792448, 0.5248833]","[0.5784415, 0.69121736]","[0.4420523, 0.70837134]","[0.5444976, 0.9180171]","[0.39973402, 0.87089473]"
1,1,1,0,"[0.9540815, 0.46759263]","[0.95418, 0.4635923]","[0.9541004, 0.46483496]","[0.9579952, 0.46755242]","[0.95550096, 0.46747252]","[0.8874654, 0.52051455]","[0.48765278, 0.2828231]","[0.92382234, 0.621996]","[0.46226963, 0.49040487]","[0.6378939, 0.4357051]","[0.45363432, 0.49722472]","[0.5422739, 0.5057809]","[0.46239334, 0.50417346]","[0.5497218, 0.6600626]","[0.43888804, 0.70858526]","[0.54252183, 0.9147122]","[0.39958978, 0.8692072]"
2,1,2,0,"[0.9552678, 0.46827152]","[0.95570683, 0.46365747]","[0.9554047, 0.46518216]","[0.9601342, 0.4666077]","[0.9571421, 0.46711835]","[0.9108266, 0.53426725]","[0.95443547, 0.5074496]","[0.9250432, 0.6167903]","[0.46285233, 0.48870206]","[0.6373829, 0.43345627]","[0.4550485, 0.49608842]","[0.5403686, 0.5027947]","[0.46016988, 0.50242424]","[0.54910856, 0.6912089]","[0.43724054, 0.70716673]","[0.5448282, 0.90966463]","[0.40227288, 0.86718255]"
3,1,3,0,"[0.9547858, 0.45990896]","[0.9542183, 0.45675418]","[0.9540833, 0.45806527]","[0.95770776, 0.464124]","[0.9556292, 0.46380022]","[0.7286011, 0.4264553]","[0.48321652, 0.27803496]","[0.5911753, 0.38876462]","[0.42691144, 0.4438524]","[0.60855865, 0.37672484]","[0.45642027, 0.49309203]","[0.5435534, 0.50146246]","[0.4595031, 0.48684168]","[0.5490419, 0.6881275]","[0.4378787, 0.70306206]","[0.54297936, 0.91354656]","[0.40170008, 0.8692108]"
4,1,4,0,"[0.9553101, 0.4638997]","[0.9550538, 0.46045852]","[0.95563334, 0.46168056]","[0.9588263, 0.46575466]","[0.9580064, 0.46532798]","[0.7312964, 0.4324056]","[0.48524952, 0.2788084]","[0.7486676, 0.45700005]","[0.46295142, 0.48813987]","[0.63608944, 0.4023483]","[0.4559425, 0.49563694]","[0.5439713, 0.5049067]","[0.46373066, 0.5038142]","[0.55132043, 0.6596282]","[0.44063866, 0.7053447]","[0.54359055, 0.9114465]","[0.3999897, 0.869935]"


In [ ]:
X_train.head()

,MoveNetLightning_nose_x,MoveNetLightning_nose_y,MoveNetLightning_left_eye_x,MoveNetLightning_left_eye_y,MoveNetLightning_right_eye_x,MoveNetLightning_right_eye_y,MoveNetLightning_left_ear_x,MoveNetLightning_left_ear_y,MoveNetLightning_right_ear_x,MoveNetLightning_right_ear_y,...,MoveNetLightning_right_hip_x,MoveNetLightning_right_hip_y,MoveNetLightning_left_knee_x,MoveNetLightning_left_knee_y,MoveNetLightning_right_knee_x,MoveNetLightning_right_knee_y,MoveNetLightning_left_ankle_x,MoveNetLightning_left_ankle_y,MoveNetLightning_right_ankle_x,MoveNetLightning_right_ankle_y
9408,0.460715,0.259938,0.460379,0.246344,0.457810,0.245723,0.440656,0.232343,0.433789,0.228475,...,0.376970,0.447249,0.378804,0.584379,0.399092,0.593078,0.367350,0.697624,0.402521,0.724599
9686,0.740000,0.190654,0.737482,0.182208,0.736859,0.181866,0.719381,0.183514,0.716841,0.182661,...,0.695687,0.386990,0.684223,0.517067,0.685869,0.525604,0.669636,0.605441,0.668806,0.627925
673,0.394430,0.612054,0.393064,0.607582,0.394435,0.608851,0.393459,0.613496,0.393264,0.615237,...,0.418599,0.703925,0.431984,0.813380,0.474087,0.790583,0.479408,0.903654,0.517291,0.894479
11186,0.550859,0.787787,0.556288,0.778540,0.552742,0.776229,0.557347,0.749489,0.547300,0.737405,...,0.363084,0.723527,0.303007,0.868982,0.285805,0.850610,0.201835,0.929291,0.174556,0.921443
3921,0.414488,0.321365,0.419843,0.312399,0.408120,0.312068,0.423948,0.315464,0.398203,0.313797,...,0.390532,0.459763,0.466406,0.399695,0.414132,0.441634,0.426906,0.487821,0.427919,0.613257


In [ ]:
X_test.head()

,MoveNetLightning_nose_x,MoveNetLightning_nose_y,MoveNetLightning_left_eye_x,MoveNetLightning_left_eye_y,MoveNetLightning_right_eye_x,MoveNetLightning_right_eye_y,MoveNetLightning_left_ear_x,MoveNetLightning_left_ear_y,MoveNetLightning_right_ear_x,MoveNetLightning_right_ear_y,...,MoveNetLightning_right_hip_x,MoveNetLightning_right_hip_y,MoveNetLightning_left_knee_x,MoveNetLightning_left_knee_y,MoveNetLightning_right_knee_x,MoveNetLightning_right_knee_y,MoveNetLightning_left_ankle_x,MoveNetLightning_left_ankle_y,MoveNetLightning_right_ankle_x,MoveNetLightning_right_ankle_y
101,0.275783,0.639305,0.274311,0.632985,0.274930,0.632384,0.284620,0.614765,0.284904,0.610699,...,0.413864,0.619515,0.476671,0.692793,0.456193,0.720534,0.561769,0.767024,0.530177,0.773444
9306,0.320923,0.214946,0.322788,0.203851,0.314586,0.203034,0.306279,0.200850,0.288884,0.197619,...,0.277329,0.416647,0.340920,0.538326,0.279621,0.546217,0.344570,0.654476,0.285600,0.655039
7189,0.803850,0.974541,0.820521,0.975398,0.811597,0.980126,0.821313,0.962336,0.802684,0.963145,...,0.701897,0.779061,0.666891,0.805544,0.639417,0.817005,0.613891,0.759452,0.575322,0.776942
9222,0.317804,0.214561,0.320902,0.203168,0.311970,0.203118,0.310938,0.197245,0.287113,0.194984,...,0.280566,0.405729,0.342292,0.527841,0.282659,0.537559,0.320047,0.620308,0.282413,0.637080
1845,0.744263,0.253682,0.749101,0.240337,0.749015,0.240710,0.779138,0.233136,0.775474,0.229408,...,0.775671,0.531392,0.753627,0.699793,0.740554,0.673123,0.756564,0.857243,0.753096,0.805362


In [ ]:
# Initialize SVM classifier
svm_model = SVC(kernel='linear', random_state=42)

# Train the SVM model
svm_model.fit(X_train, y_train)

# Predict labels on the test set
y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of SVM model: {accuracy:.2f}")


Accuracy of SVM model: 0.98


In [ ]:
# Assuming svm_model is your trained SVM model
model_filename = 'svm_lightning_model.joblib'
dump(svm_model, model_filename)

['svm_lightning_model.joblib']

#Media Pipe SVM Moddel


In [ ]:
mp_df = pd.read_csv('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/labelled_MP_keypoints.csv')

In [ ]:
MPkeypoint_columns = ['MediaPipe_nose',
       'MediaPipe_left_eye', 'MediaPipe_right_eye', 'MediaPipe_left_ear',
       'MediaPipe_right_ear', 'MediaPipe_left_shoulder',
       'MediaPipe_right_shoulder', 'MediaPipe_left_elbow',
       'MediaPipe_right_elbow', 'MediaPipe_left_wrist',
       'MediaPipe_right_wrist', 'MediaPipe_left_hip', 'MediaPipe_right_hip',
       'MediaPipe_left_knee', 'MediaPipe_right_knee', 'MediaPipe_left_ankle',
       'MediaPipe_right_ankle']
for col in MPkeypoint_columns:
    mp_df[col] = mp_df[col].apply(literal_eval)

In [ ]:
mp_df.columns

Index(['Unnamed: 0', 'video_id', 'frame_idx', 'label', 'MediaPipe_nose',
       'MediaPipe_left_eye', 'MediaPipe_right_eye', 'MediaPipe_left_ear',
       'MediaPipe_right_ear', 'MediaPipe_left_shoulder',
       'MediaPipe_right_shoulder', 'MediaPipe_left_elbow',
       'MediaPipe_right_elbow', 'MediaPipe_left_wrist',
       'MediaPipe_right_wrist', 'MediaPipe_left_hip', 'MediaPipe_right_hip',
       'MediaPipe_left_knee', 'MediaPipe_right_knee', 'MediaPipe_left_ankle',
       'MediaPipe_right_ankle'],
      dtype='object')

In [ ]:
X = mp_df[MPkeypoint_columns]
y = mp_df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
flatten_keypoints(X_train, MPkeypoint_columns)
flatten_keypoints(X_test, MPkeypoint_columns)

In [ ]:
mp_df.head()

,Unnamed: 0,video_id,frame_idx,label,MediaPipe_nose,MediaPipe_left_eye,MediaPipe_right_eye,MediaPipe_left_ear,MediaPipe_right_ear,MediaPipe_left_shoulder,...,MediaPipe_left_elbow,MediaPipe_right_elbow,MediaPipe_left_wrist,MediaPipe_right_wrist,MediaPipe_left_hip,MediaPipe_right_hip,MediaPipe_left_knee,MediaPipe_right_knee,MediaPipe_left_ankle,MediaPipe_right_ankle
0,0,1,0,0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
1,1,1,1,0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
2,2,1,2,0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
3,3,1,3,0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
4,4,1,4,0,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"


In [ ]:
X_train.head()

,MediaPipe_nose_x,MediaPipe_nose_y,MediaPipe_left_eye_x,MediaPipe_left_eye_y,MediaPipe_right_eye_x,MediaPipe_right_eye_y,MediaPipe_left_ear_x,MediaPipe_left_ear_y,MediaPipe_right_ear_x,MediaPipe_right_ear_y,...,MediaPipe_right_hip_x,MediaPipe_right_hip_y,MediaPipe_left_knee_x,MediaPipe_left_knee_y,MediaPipe_right_knee_x,MediaPipe_right_knee_y,MediaPipe_left_ankle_x,MediaPipe_left_ankle_y,MediaPipe_right_ankle_x,MediaPipe_right_ankle_y
9408,0.461134,0.254035,0.460694,0.240234,0.457995,0.239040,0.444316,0.232520,0.439126,0.231174,...,0.381420,0.467536,0.386725,0.578048,0.396718,0.597261,0.371197,0.675488,0.398516,0.716449
9686,0.731624,0.192848,0.729060,0.182360,0.726785,0.182630,0.720101,0.183528,0.713629,0.183785,...,0.690133,0.399672,0.691388,0.502019,0.681830,0.518402,0.669586,0.598421,0.664933,0.624861
673,0.390004,0.640870,0.384404,0.638353,0.396212,0.635395,0.379403,0.643076,0.404920,0.633847,...,0.443177,0.712492,0.432214,0.798210,0.461411,0.807658,0.465487,0.895264,0.517316,0.893183
11186,0.521442,0.780472,0.529354,0.768693,0.528785,0.761640,0.524596,0.752891,0.522456,0.740102,...,0.361446,0.719424,0.310832,0.889825,0.296096,0.834447,0.204645,0.923515,0.180475,0.925378
3921,0.414746,0.319269,0.417795,0.311982,0.408076,0.311224,0.419661,0.314472,0.399674,0.312281,...,0.398092,0.463595,0.469390,0.400244,0.409085,0.479513,0.407425,0.446363,0.426599,0.602493


In [ ]:
X_test.head()

,MediaPipe_nose_x,MediaPipe_nose_y,MediaPipe_left_eye_x,MediaPipe_left_eye_y,MediaPipe_right_eye_x,MediaPipe_right_eye_y,MediaPipe_left_ear_x,MediaPipe_left_ear_y,MediaPipe_right_ear_x,MediaPipe_right_ear_y,...,MediaPipe_right_hip_x,MediaPipe_right_hip_y,MediaPipe_left_knee_x,MediaPipe_left_knee_y,MediaPipe_right_knee_x,MediaPipe_right_knee_y,MediaPipe_left_ankle_x,MediaPipe_left_ankle_y,MediaPipe_right_ankle_x,MediaPipe_right_ankle_y
101,0.281900,0.631489,0.277422,0.619984,0.278194,0.619195,0.283910,0.607916,0.286054,0.606867,...,0.414433,0.643829,0.480535,0.697211,0.462045,0.724632,0.557106,0.760612,0.529306,0.782962
9306,0.321390,0.213546,0.321735,0.201562,0.315600,0.199630,0.308766,0.196412,0.297503,0.192787,...,0.276385,0.425810,0.336309,0.544174,0.286047,0.545383,0.340961,0.654585,0.292163,0.641294
7189,0.757629,0.992055,0.766423,0.998248,0.766026,0.986530,0.786010,0.982725,0.775879,0.963403,...,0.703037,0.784442,0.646967,0.809588,0.629308,0.819549,0.615234,0.743065,0.589709,0.771382
9222,0.316597,0.214441,0.319297,0.203429,0.310850,0.202390,0.309639,0.197164,0.293469,0.191787,...,0.277966,0.416078,0.336849,0.528093,0.286260,0.540971,0.319250,0.615473,0.289972,0.637268
1845,0.752414,0.263308,0.754422,0.247957,0.755369,0.247645,0.768453,0.244679,0.770754,0.242294,...,0.768561,0.547808,0.756476,0.714091,0.745543,0.679478,0.750365,0.854878,0.750099,0.813511


In [ ]:
mp_svm_model = SVC(kernel='linear', random_state=42)

# Train the SVM model
mp_svm_model.fit(X_train, y_train)

# Predict labels on the test set
y_pred = mp_svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of SVM model: {accuracy:.2f}")


Accuracy of SVM model: 0.97


In [ ]:
model_filename = 'svm_MediaPipe_model.joblib'
dump(mp_svm_model, model_filename)

['svm_MediaPipe_model.joblib']

#Neural Network Model (MLP)

In [ ]:
mp_df = pd.read_csv('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/labelled_MP_keypoints.csv')
MPkeypoint_columns = ['MediaPipe_nose',
       'MediaPipe_left_eye', 'MediaPipe_right_eye', 'MediaPipe_left_ear',
       'MediaPipe_right_ear', 'MediaPipe_left_shoulder',
       'MediaPipe_right_shoulder', 'MediaPipe_left_elbow',
       'MediaPipe_right_elbow', 'MediaPipe_left_wrist',
       'MediaPipe_right_wrist', 'MediaPipe_left_hip', 'MediaPipe_right_hip',
       'MediaPipe_left_knee', 'MediaPipe_right_knee', 'MediaPipe_left_ankle',
       'MediaPipe_right_ankle']
for col in MPkeypoint_columns:
    mp_df[col] = mp_df[col].apply(lambda x: np.array(eval(x)))
    mp_df[col] =  mp_df[col].to_list()

mv_df = pd.read_csv('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/labeled_keypointsWithoutScore.csv')
MvLight_keypoint_columns = ['MoveNetLightning_nose', 'MoveNetLightning_left_eye', 'MoveNetLightning_right_eye',
                    'MoveNetLightning_left_ear', 'MoveNetLightning_right_ear',
                    'MoveNetLightning_left_shoulder', 'MoveNetLightning_right_shoulder',
                    'MoveNetLightning_left_elbow', 'MoveNetLightning_right_elbow',
                    'MoveNetLightning_left_wrist', 'MoveNetLightning_right_wrist',
                    'MoveNetLightning_left_hip', 'MoveNetLightning_right_hip',
                    'MoveNetLightning_left_knee', 'MoveNetLightning_right_knee',
                    'MoveNetLightning_left_ankle', 'MoveNetLightning_right_ankle']

for col in MvLight_keypoint_columns:
    mv_df[col] = mv_df[col].apply(lambda x: np.array(eval(x)))
    mv_df[col] =  mv_df[col].to_list()


In [ ]:
# Define your features and target
X = mp_df[MPkeypoint_columns]
y = mp_df['label'].values


In [ ]:
# Define your features and target
MvX = mv_df[MvLight_keypoint_columns]
Mvy = mv_df['label'].values

In [ ]:
data = np.array([X.iloc[i].values.tolist() for i in range(len(X))])
Mvdata = np.array([MvX.iloc[i].values.tolist() for i in range(len(MvX))])

In [ ]:
print("Order of keypoints in the data matches the specified keypoint columns:",
      all(MvX.columns == pd.Index(MvLight_keypoint_columns)))

Order of keypoints in the data matches the specified keypoint columns: True


In [ ]:
print(data.shape)
data[20]

(11723, 17, 2)


array([[0.86204416, 0.36424387],
       [0.86631   , 0.34931174],
       [0.86804658, 0.34693295],
       [0.87903249, 0.35195437],
       [0.88298005, 0.34957844],
       [0.8829211 , 0.45475781],
       [0.93916905, 0.42680556],
       [0.87580889, 0.60690373],
       [0.93292588, 0.56662071],
       [0.85077858, 0.70568043],
       [0.90197533, 0.64456737],
       [0.88191605, 0.68799108],
       [0.92560256, 0.68372029],
       [0.82282382, 0.80298966],
       [0.92916501, 0.84100431],
       [0.76833063, 0.91321862],
       [0.94629574, 0.99345005]])

In [ ]:
# Step 1: Define the model with TimeDistributed layer
model = Sequential([
    Flatten(input_shape=(17, 2)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to check the architecture
model.summary()

# Step 2: Train the model
model.fit(data, y, epochs=50, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 34)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           4,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,801 (50.00 KB)

 Trainable params: 12,801 (50.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8602 - loss: 0.3464 - val_accuracy: 0.9441 - val_loss: 0.1704
Epoch 2/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9600 - loss: 0.1159 - val_accuracy: 0.9416 - val_loss: 0.1601
Epoch 3/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9579 - loss: 0.1132 - val_accuracy: 0.9070 - val_loss: 0.1734
Epoch 4/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9635 - loss: 0.0999 - val_accuracy: 0.9441 - val_loss: 0.1475
Epoch 5/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9674 - loss: 0.0947 - val_accuracy: 0.9058 - val_loss: 0.1676
Epoch 6/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9645 - loss: 0.0960 - val_accuracy: 0.9066 - val_loss: 0.1909
Epoch 7/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9675 - loss: 0.0914 - val_accuracy: 0.9066 - val_loss: 0.2115
Epoch 8/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9663 - loss: 0.0965 - val_accuracy: 0.

In [ ]:
# Step 1: Define the model with TimeDistributed layer
Mvmodel = Sequential([
    Flatten(input_shape=(17, 2)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

Mvmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to check the architecture
Mvmodel.summary()

# Step 2: Train the model
Mvmodel.fit(Mvdata, Mvy, epochs=50, batch_size=32, validation_split=0.2)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 34)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           4,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,801 (50.00 KB)

 Trainable params: 12,801 (50.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8539 - loss: 0.3796 - val_accuracy: 0.9825 - val_loss: 0.1112
Epoch 2/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9462 - loss: 0.1686 - val_accuracy: 0.9744 - val_loss: 0.0987
Epoch 3/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9507 - loss: 0.1487 - val_accuracy: 0.9757 - val_loss: 0.0930
Epoch 4/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9546 - loss: 0.1369 - val_accuracy: 0.9778 - val_loss: 0.0843
Epoch 5/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9577 - loss: 0.1274 - val_accuracy: 0.9795 - val_loss: 0.0746
Epoch 6/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9625 - loss: 0.1231 - val_accuracy: 0.9804 - val_loss: 0.0754
Epoch 7/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9584 - loss: 0.1184 - val_accuracy: 0.9812 - val_loss: 0.0719
Epoch 8/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9640 - loss: 0.1107 - val_accuracy: 0.

In [ ]:
#@title Check the weights
# Access the weights of the first Dense layer
dense1_weights = model.layers[1].get_weights()[0]  # The first Dense layer after Flatten

print(dense1_weights.shape)  # Should print (34, 128)

# Sum the weights for each of the 17 input features
weights_sum = np.sum(np.abs(dense1_weights), axis=1)  # Summing the absolute values of weights

# Reshape to get the sum for each pair (17 features)
weights_sum_reshaped = weights_sum.reshape(17, 2).sum(axis=1)

# Get the indices of the features sorted by their weight sum
sorted_indices = np.argsort(weights_sum_reshaped)[::-1]  # Descending order

# Print the feature names sorted by their weight importance
sorted_features = [MPkeypoint_columns[i] for i in sorted_indices]
print("Features sorted by their weight importance:", sorted_features)

# Print the corresponding weight sums for each feature
print("Corresponding weight sums:", weights_sum_reshaped[sorted_indices])


(34, 128)
Features sorted by their weight importance: ['MediaPipe_left_wrist', 'MediaPipe_right_ankle', 'MediaPipe_left_ankle', 'MediaPipe_right_shoulder', 'MediaPipe_right_ear', 'MediaPipe_left_shoulder', 'MediaPipe_right_hip', 'MediaPipe_left_knee', 'MediaPipe_right_elbow', 'MediaPipe_left_eye', 'MediaPipe_right_eye', 'MediaPipe_right_wrist', 'MediaPipe_left_elbow', 'MediaPipe_right_knee', 'MediaPipe_left_ear', 'MediaPipe_nose', 'MediaPipe_left_hip']
Corresponding weight sums: [35.232952 32.47973  32.306637 31.533802 31.354483 30.836287 30.56968
 30.133337 29.528934 28.85816  28.782267 28.771582 28.501022 28.398582
 28.126945 27.307444 26.457458]


In [ ]:

Mvmodel.save('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/MoveNet_NN_model50Epochsv2.keras')
model.save('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/MediaPipe_NN_model50Epochsv2.keras')

#Neural Network Model (LSTM)

In [ ]:
mp_df = pd.read_csv('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/labelled_MP_keypoints.csv')
MPkeypoint_columns = ['MediaPipe_nose', 'MediaPipe_left_eye', 'MediaPipe_right_eye', 'MediaPipe_left_ear',
                      'MediaPipe_right_ear', 'MediaPipe_left_shoulder', 'MediaPipe_right_shoulder',
                      'MediaPipe_left_elbow', 'MediaPipe_right_elbow', 'MediaPipe_left_wrist',
                      'MediaPipe_right_wrist', 'MediaPipe_left_hip', 'MediaPipe_right_hip',
                      'MediaPipe_left_knee', 'MediaPipe_right_knee', 'MediaPipe_left_ankle',
                      'MediaPipe_right_ankle']
for col in MPkeypoint_columns:
    mp_df[col] = mp_df[col].apply(lambda x: np.array(eval(x)))
    mp_df[col] = mp_df[col].to_list()

mv_df = pd.read_csv('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/labeled_keypointsWithoutScore.csv')
MvLight_keypoint_columns = ['MoveNetLightning_nose', 'MoveNetLightning_left_eye', 'MoveNetLightning_right_eye',
                            'MoveNetLightning_left_ear', 'MoveNetLightning_right_ear',
                            'MoveNetLightning_left_shoulder', 'MoveNetLightning_right_shoulder',
                            'MoveNetLightning_left_elbow', 'MoveNetLightning_right_elbow',
                            'MoveNetLightning_left_wrist', 'MoveNetLightning_right_wrist',
                            'MoveNetLightning_left_hip', 'MoveNetLightning_right_hip',
                            'MoveNetLightning_left_knee', 'MoveNetLightning_right_knee',
                            'MoveNetLightning_left_ankle', 'MoveNetLightning_right_ankle']
for col in MvLight_keypoint_columns:
    mv_df[col] = mv_df[col].apply(lambda x: np.array(eval(x)))
    mv_df[col] = mv_df[col].to_list()


In [ ]:
# Define features and targets
X = mp_df[MPkeypoint_columns]
y = mp_df['label'].values

MvX = mv_df[MvLight_keypoint_columns]
Mvy = mv_df['label'].values

data = np.array([X.iloc[i].values.tolist() for i in range(len(X))])
Mvdata = np.array([MvX.iloc[i].values.tolist() for i in range(len(MvX))])


In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
MvX_train, MvX_test, Mvy_train, Mvy_test = train_test_split(Mvdata, Mvy, test_size=0.2, random_state=42)


In [ ]:

# Define the LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(17, 2)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

mp_model = create_lstm_model()
mv_model = create_lstm_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Print the model summary
mp_model.summary()
mv_model.summary()

# Train the model
mp_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
mv_model.fit(MvX_train, Mvy_train, epochs=50, batch_size=32, validation_data=(MvX_test, Mvy_test))


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50)                  │          10,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,651 (41.61 KB)

 Trainable params: 10,651 (41.61 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 50)                  │          10,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,651 (41.61 KB)

 Trainable params: 10,651 (41.61 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.8186 - loss: 0.3849 - val_accuracy: 0.9454 - val_loss: 0.1238
Epoch 2/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9431 - loss: 0.1549 - val_accuracy: 0.9565 - val_loss: 0.1138
Epoch 3/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9535 - loss: 0.1233 - val_accuracy: 0.9595 - val_loss: 0.1022
Epoch 4/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9523 - loss: 0.1213 - val_accuracy: 0.9586 - val_loss: 0.1011
Epoch 5/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9537 - loss: 0.1204 - val_accuracy: 0.9591 - val_loss: 0.1057
Epoch 6/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9522 - loss: 0.1264 - val_accuracy: 0.9497 - val_loss: 0.1198
Epoch 7/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9567 - loss: 0.1206 - val_accuracy: 0.9548 - val_loss: 0.1102
Epoch 8/50
294/294 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9549 - loss: 0.1169 - val_accurac

In [ ]:
# Save the models
mp_model.save('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/lstm_fall_detection_mpv2.h5')
mv_model.save('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/lstm_fall_detection_mvv2.h5')
print("Models saved successfully.")


Models saved successfully.


In [ ]:
# Load the models
loaded_mp_model = tf.keras.models.load_model('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/lstm_fall_detection_mpv2.h5')
loaded_mv_model = tf.keras.models.load_model('/content/drive/MyDrive/AI-RDP/DETECTION MODELS/lstm_fall_detection_mvv2.h5')
print("Models loaded successfully.")

Models loaded successfully.


In [ ]:

# Verify the loaded models
mp_loss, mp_accuracy = loaded_mp_model.evaluate(X_test, y_test)
print(f"Loaded MP model accuracy: {mp_accuracy:.2f}")
mv_loss, mv_accuracy = loaded_mv_model.evaluate(MvX_test, Mvy_test)
print(f"Loaded MV model accuracy: {mv_accuracy:.2f}")


74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9661 - loss: 0.0901
Loaded MP model accuracy: 0.96
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9752 - loss: 0.0849
Loaded MV model accuracy: 0.97


In [ ]:
print(f'The Models were trained and saved using this version of keras: {tf.keras.__version__}')

The Models were trained and saved using this version of keras: 3.4.1
